In [1]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1746379942544_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
PATH = 's3://p8-data-olivier-test'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/ResultsTest'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p8-data-olivier-test
PATH_Data:   s3://p8-data-olivier-test/Test
PATH_Result: s3://p8-data-olivier-test/ResultsTest

In [3]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-data-oliv...|2025-04-29 15:34:58|125135|[FF D8 FF E0 00 1...|
|s3://p8-data-oliv...|2025-04-29 15:34:58|124785|[FF D8 FF E0 00 1...|
|s3://p8-data-oliv...|2025-04-29 15:34:58|123514|[FF D8 FF E0 00 1...|
|s3://p8-data-oliv...|2025-04-29 15:34:58|122958|[FF D8 FF E0 00 1...|
|s3://p8-data-oliv...|2025-04-29 15:34:58|122807|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [5]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------+-----------+
|path                                                 |label      |
+-----------------------------------------------------+-----------+
|s3://p8-data-olivier-test/Test/apple_hit_1/r0_115.jpg|apple_hit_1|
|s3://p8-data-olivier-test/Test/apple_hit_1/r0_119.jpg|apple_hit_1|
|s3://p8-data-olivier-test/Test/apple_hit_1/r0_107.jpg|apple_hit_1|
|s3://p8-data-olivier-test/Test/apple_hit_1/r0_143.jpg|apple_hit_1|
|s3://p8-data-olivier-test/Test/apple_hit_1/r0_111.jpg|apple_hit_1|
+-----------------------------------------------------+-----------+
only showing top 5 rows

None

In [6]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [7]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [10]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [12]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# UDF pour convertir array<float> en DenseVector
to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())

# Appliquer l'UDF pour créer une vraie colonne Vector
features_vec_df = features_df.withColumn("features_vec", to_vector_udf("features"))

# PCA
pca = PCA(k=50, inputCol="features_vec", outputCol="pca_features")
pca_model = pca.fit(features_vec_df)
df_pca = pca_model.transform(features_vec_df)

# Afficher les résultats
df_pca.select("path", "pca_features").show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://p8-data-olivier-test/ResultsTest

In [15]:
df_pca.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df["pca_features"] = df["pca_features"].apply(lambda x: x["values"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
print(df["pca_features"].iloc[0][:5])
print(df["pca_features"].apply(len).value_counts())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[ 6.62137688  1.52071622 -3.90790188  2.95442854  7.69691863]
50    3110
Name: pca_features, dtype: int64

In [20]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                       pca_features
0  s3://p8-data-olivier-test/Test/apple_hit_1/r2_...  ...  [6.621376876733395, 1.520716216243603, -3.9079...
1  s3://p8-data-olivier-test/Test/apple_hit_1/r2_...  ...  [6.3487555088584475, 1.0596641411277115, -4.38...
2  s3://p8-data-olivier-test/Test/cabbage_white_1...  ...  [4.636196895152497, -4.2998316515356905, 1.314...
3  s3://p8-data-olivier-test/Test/cucumber_3/r0_1...  ...  [4.428504123549715, -4.132795379950242, -4.551...
4  s3://p8-data-olivier-test/Test/apple_red_3/r1_...  ...  [-9.271137806659887, 1.5933279318045066, -4.75...

[5 rows x 5 columns]

In [21]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3110, 5)

In [33]:
from pyspark.sql.functions import col

# Convertir le vecteur en tableau Python
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType

vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))
df_csv_ready = df_pca.select("path", vector_to_array("pca_features").alias("pca_array"))

# Aplatir le tableau en colonnes pca_0, ..., pca_49
for i in range(50):
    df_csv_ready = df_csv_ready.withColumn(f"pca_{i}", col("pca_array")[i])

# Supprimer la colonne pca_array
df_csv_ready = df_csv_ready.drop("pca_array")


# Ecrire en CSV
df_csv_ready.write.mode("overwrite").option("header", True).csv("s3://p8-data-olivier-test/ResultsCSV")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…